# Nonimal data experiment
* [ER-MLP](#ER-MLP)
* [NTN](#NTN)

In [1]:
from core.knowledge_graph import KnowledgeGraph
from core.link_predict_utils import *

import tensorflow as tf
%pylab nbagg --no-import-all

%load_ext autoreload
%autoreload 2

test_percent = 0.1
database = KnowledgeGraph()
database.read_data_from_txt('data/kin_nominal')
database.spilt_train_valid_test(0, test_percent)
num_entities = database.number_of_entities()
num_relations = database.number_of_relations()

Populating the interactive namespace from numpy and matplotlib


## ER-MLP

In [6]:
with tf.Graph().as_default(),tf.Session() as sess:
    saver = tf.train.import_meta_graph('saved_model/er_mlp_model-kin_nominal.meta')
    saver.restore(sess, 'saved_model/er_mlp_model-kin_nominal')

    batch = sess.graph.get_tensor_by_name('Placeholders/batch:0')
    predicts = sess.graph.get_tensor_by_name('Output/predicts:0')
    
    dataset = database.get_all_triples()
    hh = []
    for tripe in dataset:
        if tripe[2] == 0 or tripe[2] == 1:
            hh.append(tripe)
    sspredicts = sess.run(predicts, {batch: dataset})
    print(sum(sspredicts>0.5))
    ture_c = 0
    for i in range(23, 127):
        test0 = []
        test1 = []
        for j in range(23):
            test0.append((i,j,0))
            test1.append((i,j,1))

        test_predicts = sess.run(predicts, {batch: test0})
        
        height = np.argsort(-1 * test_predicts)
        ture_c = ture_c + database.check_triple((i,height[0],0))
        print(database.check_triple((i,height[0],0)))
        print('%s has height %s, confidence %f %f' % (
               database.entity_index_to_name(i), database.entity_index_to_name(height[0]),
               test_predicts[height[0]], test_predicts[height[1]]))

        test_predicts = sess.run(predicts, {batch: test1})

        height = np.argsort(-1 * test_predicts)
        print('%s has stature %s, confidence %f %f\n' % (
               database.entity_index_to_name(i), database.entity_index_to_name(height[0]),
               test_predicts[height[0]], test_predicts[height[1]]))
    print(ture_c / 104)

INFO:tensorflow:Restoring parameters from saved_model/er_mlp_model-kin_nominal
10666
True
Person0 has height 172_cm, confidence 0.987667 0.121743
Person0 has stature 172_cm, confidence 0.883619 0.255494

True
Person1 has height 175_cm, confidence 0.994552 0.310350
Person1 has stature 171_cm, confidence 0.663167 0.377706

True
Person2 has height 174_cm, confidence 0.992685 0.055234
Person2 has stature 174_cm, confidence 0.603811 0.088012

True
Person3 has height 162_cm, confidence 0.992361 0.143761
Person3 has stature 162_cm, confidence 0.994024 0.832865

True
Person4 has height 166_cm, confidence 0.990882 0.178857
Person4 has stature 166_cm, confidence 0.850176 0.612835

True
Person5 has height 179_cm, confidence 0.992193 0.204173
Person5 has stature 179_cm, confidence 0.938150 0.174014

True
Person6 has height 177_cm, confidence 0.993984 0.816377
Person6 has stature taller_than_180_cm, confidence 0.940537 0.908634

True
Person7 has height 166_cm, confidence 0.988828 0.624856
Person7 h

Person73 has stature 168_cm, confidence 0.287469 0.087821

True
Person74 has height 168_cm, confidence 0.992405 0.467482
Person74 has stature 168_cm, confidence 0.597560 0.405602

True
Person75 has height 180_cm, confidence 0.986816 0.467659
Person75 has stature 180_cm, confidence 0.099842 0.010432

True
Person76 has height 175_cm, confidence 0.991326 0.547464
Person76 has stature 175_cm, confidence 0.516122 0.291054

False
Person77 has height 170_cm, confidence 0.988748 0.983641
Person77 has stature 170_cm, confidence 0.997612 0.986269

True
Person78 has height 177_cm, confidence 0.993603 0.175009
Person78 has stature 177_cm, confidence 0.291454 0.182888

True
Person79 has height 163_cm, confidence 0.987811 0.362692
Person79 has stature 163_cm, confidence 0.396142 0.152967

True
Person80 has height 162_cm, confidence 0.986470 0.340658
Person80 has stature 162_cm, confidence 0.986007 0.729615

True
Person81 has height 160_cm, confidence 0.993992 0.318757
Person81 has stature 160_cm, co

## NTN

In [4]:
def fill_feed_dict(input_list_r, data_list_r, input_list, data_list, num_relations):
    feed_dict = {}
    for (var, value) in zip(input_list_r, data_list_r):
        for r in range(num_relations):
            feed_dict[var[r]] = value[r]

    for (var, value) in zip(input_list, data_list):
        feed_dict[var] = value

    return feed_dict

In [5]:
with tf.Graph().as_default(), tf.Session() as sess:
    saver = tf.train.import_meta_graph('saved_model/ntn_model-kin_nominal.meta')
    saver.restore(sess, 'saved_model/ntn_model-kin_nominal')

    num_relations = database.number_of_relations()
    batch_input = [sess.graph.get_tensor_by_name('Placeholders/batch_%d:0' % r)
                   for r in range(num_relations)]
    r_empty_input = [sess.graph.get_tensor_by_name('Placeholders/empty_%d:0' % r)
                     for r in range(num_relations)]
    predicts = sess.graph.get_tensor_by_name('Output/predicts:0')
    
    ture_c = 0
    for i in range(23, 127):
        test0 = []
        test1 = []
        for j in range(127):
            test0.append((i,j,0))
            test1.append((i,j,1))

        test_data, _, r_empty= make_split(test0, [1]*len(test0), num_relations)
        feed_dict = fill_feed_dict([batch_input, r_empty_input],
                                   [test_data, r_empty], [], [], num_relations)
        test_predicts = sess.run(predicts, feed_dict)
        print(database.check_triple((i,height[0],0)))
        height = np.argsort(-1 * test_predicts)
        ture_c = ture_c + database.check_triple((i,height[0],0))
        print('%s has height %s, confidence %f %f' % (
               database.entity_index_to_name(i), database.entity_index_to_name(height[0]),
               test_predicts[height[0]], test_predicts[height[1]]))

        test_data, _, r_empty= make_split(test1, [1]*len(test1), num_relations)
        feed_dict = fill_feed_dict([batch_input, r_empty_input],
                                   [test_data, r_empty], [], [], num_relations)
        test_predicts = sess.run(predicts, feed_dict)

        height = np.argsort(-1 * test_predicts)
        print('%s has stature %s, confidence %f %f\n' % (
               database.entity_index_to_name(i), database.entity_index_to_name(height[0]),
               test_predicts[height[0]], test_predicts[height[1]]))
    print(ture_c / 104)

INFO:tensorflow:Restoring parameters from saved_model/ntn_model-kin_nominal
False
Person0 has height 172_cm, confidence 0.547715 0.545171
Person0 has stature 171_cm, confidence 0.501529 0.433968

False
Person1 has height shorter_than_160_cm, confidence 0.529955 0.525783
Person1 has stature 171_cm, confidence 0.495675 0.435473

False
Person2 has height shorter_than_160_cm, confidence 0.523633 0.503910
Person2 has stature 167_cm, confidence 0.451384 0.449598

False
Person3 has height shorter_than_160_cm, confidence 0.542217 0.481659
Person3 has stature shorter_than_160_cm, confidence 0.450596 0.427235

False
Person4 has height shorter_than_160_cm, confidence 0.505722 0.456345
Person4 has stature 167_cm, confidence 0.458077 0.442415

False
Person5 has height shorter_than_160_cm, confidence 0.533345 0.530803
Person5 has stature 171_cm, confidence 0.501292 0.438699

False
Person6 has height shorter_than_160_cm, confidence 0.498237 0.468833
Person6 has stature 167_cm, confidence 0.426099 0.4

Person65 has stature 167_cm, confidence 0.437655 0.427800

False
Person66 has height shorter_than_160_cm, confidence 0.499453 0.496748
Person66 has stature shorter_than_160_cm, confidence 0.425889 0.408334

False
Person67 has height 171_cm, confidence 0.565552 0.524392
Person67 has stature 171_cm, confidence 0.477641 0.404972

False
Person68 has height shorter_than_160_cm, confidence 0.485362 0.471331
Person68 has stature 167_cm, confidence 0.422207 0.418566

False
Person69 has height 175_cm, confidence 0.524487 0.510782
Person69 has stature shorter_than_160_cm, confidence 0.419916 0.398514

False
Person70 has height 172_cm, confidence 0.529102 0.490857
Person70 has stature 172_cm, confidence 0.435004 0.427052

False
Person71 has height 175_cm, confidence 0.537595 0.518948
Person71 has stature shorter_than_160_cm, confidence 0.423901 0.403153

False
Person72 has height 175_cm, confidence 0.517100 0.499239
Person72 has stature shorter_than_160_cm, confidence 0.425678 0.404022

False
Per